# Análise de Médias Móveis de um Ativo
Análise de médias móveis e tendência dos preços de um ativo financeiro.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as pdr
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
from scipy.stats import norm

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acoes = "MSFT"
Periodo_Inicio = datetime.datetime(2018,6,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
Dados = pdr.get_data_yahoo(Acoes, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise dos Preços
Candlestick da série de preços do ativo.

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados.Close))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados.Open,
                                          high=Dados.High,
                                          low=Dados.Low,
                                          close=Dados.Close))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

- Janela de tempo para uso das Médias Móveis

In [ ]:
JanelaDeTempo = 21

### Média Móvel Simples (MA)
Cálculo da série de médias móveis simples.<br />
Com a série de MA é possível avaliar a tendência de alta ou de queda do ativo, dando um mesmo peso para os preços na média.

In [ ]:
MA = Dados.Close.rolling(window=JanelaDeTempo).mean()

In [ ]:
SerieComMediaMovel = plg.Figure()
SerieComMediaMovel.add_trace(plg.Scatter(x=MA.index,
                                        y=MA,
                                        name="MA(21)",
                                        opacity=0.5))
SerieComMediaMovel.add_trace(plg.Candlestick(x=Dados.index,
                                            open=Dados.Open,
                                            high=Dados.High,
                                            low=Dados.Low,
                                            close=Dados.Close,
                                            name="Preços"))
Plot_Show(SerieComMediaMovel, "Preços & Média Móvel")

### Média Móvel Exponencial (EMA)
Cálculo da série de médias móveis exponencial.<br />
Com a série de EMA é possível avaliar a tendência de alta ou de queda do ativo, dando um peso mais alto para os preços mais recentes.

In [ ]:
FatorK = (2 / (JanelaDeTempo+1))

In [ ]:
MA = Dados.Close.rolling(window=JanelaDeTempo).mean().dropna()

In [ ]:
EMA = pd.DataFrame(index=MA.index)
EMA["Preco"] = Dados.Close
EMA["MA"] = MA
EMA["EMA"] = np.NaN

In [ ]:
EMA.EMA[0] = EMA.MA[1]

for i in range(1, len(EMA)):
    EMA.EMA[i] = (EMA.Preco[i] * FatorK) + ((1-FatorK) * EMA.EMA[i-1])

In [ ]:
SerieComMediaMovel = plg.Figure()
SerieComMediaMovel.add_trace(plg.Scatter(x=MA.index,
                                        y=MA,
                                        name="MA(21)",
                                        opacity=0.5))
SerieComMediaMovel.add_trace(plg.Scatter(x=EMA.index,
                                        y=EMA.EMA,
                                        name="EMA(21)",
                                        opacity=0.5))
SerieComMediaMovel.add_trace(plg.Candlestick(x=Dados.index,
                                            open=Dados.Open,
                                            high=Dados.High,
                                            low=Dados.Low,
                                            close=Dados.Close,
                                            name="Preços"))
Plot_Show(SerieComMediaMovel, "Preços & Média Móvel")

### Cruzamento de Médias Móveis
Análise do cruzamento das médias móveis é uma técnica para decisão de compra ou venda de um ativo.
- Média curta cruzando a longa para cima = Sinal para Compra.
- Média curta cruzando a longa para baixo = Sinal para Venda.

In [ ]:
JanelaDeTempoShort = 9
JanelaDeTempoLong  = 21

In [ ]:
MA_Short = Dados.Close.rolling(window=JanelaDeTempoShort).mean()

In [ ]:
MA_Long = Dados.Close.rolling(window=JanelaDeTempoLong).mean()

In [ ]:
SerieComMediaMovel = plg.Figure()
SerieComMediaMovel.add_trace(plg.Scatter(x=MA_Short.index,
                                        y=MA_Short,
                                        name="MA(9)",
                                        opacity=0.5))
SerieComMediaMovel.add_trace(plg.Scatter(x=MA_Long.index,
                                        y=MA_Long,
                                        name="MA(21)",
                                        opacity=0.5))
SerieComMediaMovel.add_trace(plg.Candlestick(x=Dados.index,
                                            open=Dados.Open,
                                            high=Dados.High,
                                            low=Dados.Low,
                                            close=Dados.Close,
                                            name="Preços"))
Plot_Show(SerieComMediaMovel, "Preços & Média Móvel")